In [367]:
import os
import pandas as pd
import fnmatch
import re

In [368]:
# BIGGO TO DO:
# trash bin for each filter

In [369]:
# NOTE: Possible names for each sequence

# "3D_T1": ['*SAG*T1*NII', '*SAG*FSPGR*3D*NII']
# "FLAIR": <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
# "BOLD": <ID>_BOLD_<DOE>.nii
# Diffusion
# "ASL": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii
# "SWAN": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii
# "Carotid"
# "Aorta": <ID>_[CINE]_AORTA_VENC_<number>_<DOE>.nii

In [370]:
def get_sequence_info(sequence_name):
    return (sequence_name.replace('-','_').replace('.','_').split('_'))[:-1]

#selected everything expect the '.nii' part of the string

In [371]:
def base_standard(o_name):
    n_name = o_name.replace('SAGITTAL', 'SAG').replace('CUBE', '3D').replace('AXIAL', 'AX').replace('CORONAL', 'COR')
    
    info=get_sequence_info(n_name)
    
    ID = info[0]
    # ID does not follow designation [2-3 letter cohort designation][3-4 number designation]
    match = re.match(r"([A-Z]+)([0-9]+)", ID, re.I)
    if match:
        parts = match.groups()
        if parts[0] == 'WHITTIER':
            n_name = n_name.replace('WHITTIER', 'WH')
    
    return n_name

In [372]:
def get_DOE(info):
    if 'I' in info[-1]:
        DOE=info[-2]
        return(DOE[0:4]+"/"+DOE[4:6]+'/'+DOE[6:8])
    else:
        DOE=info[-1]
        return(DOE[0:4]+"/"+DOE[4:6]+'/'+DOE[6:8])

In [373]:
def ignore_notes(sequence_info):
    other = []
    
    for part in sequence_info:
        if '(' not in part and ')' not in part:
            other.append(part)

    return other

In [374]:
def get_plane(info):
    if is_3D(info):
        if 'SAG' in info: return 'FSPGR_3D_SAG'
        elif 'COR' in info: return 'FSPGR_3D_COR'
        elif 'AX' in info: return 'FSPGR_3D_AX'
        else: return 'UNKNOWN'
    else:
        if 'SAG' in info: return 'FSPGR_SAG'
        elif 'COR' in info: return 'FSPGR_COR'
        elif 'AX' in info: return 'FSPGR_AX'
        else: return 'UNKNOWN'

In [375]:
def filt_T1(file_name):
    # TO-DO: how to handle 
    # 1) NEUROMELANIN scans
    # 2) SE_MT
    # 3) BRAVO
    # 4) 2DTOF
    # 5) repeat?
    
    sequence_info=get_sequence_info(file_name)
    
    if 'I' in sequence_info[-1]:
        return None

    ignore=['AVG', 'MIN', 'POST_PROCESS']
    if any(word in file_name for word in ignore):
        return None
    
    accept = ['SAG', 'AX', 'COR']
    if sequence_info[1] not in accept:
        return None
    
#     print("file name:", file_name)
    return file_name[:-4]+'.nii'

    
#TO-DO: repeated exam of the same date or at the different date
    

In [480]:
# Naming to be estabilshed
# <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_[MIP_ | MIN_IP_]<DOE>[_<tag>].nii
def filt_FLAIR(sequence_name, flair_3d, flair_2d):
    sequence_info=get_sequence_info(sequence_name)
            
    ignore=['AVG', 'MIN', 'MIP']
    if any(word in sequence_name for word in ignore):
        return None
    
    if "SAG" in sequence_name:
#         print(sequence_info)
        if is_3D(sequence_name):
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    elif 'AX' in sequence_name:
#         print(sequence_info)
        if is_3D(sequence_name):
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    elif 'COR' in sequence_name:
#         print(sequence_info)
        if is_3D(sequence_name):
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    else: 
#         print("\t ", sequence_name)
        return None
            
    sequence_name = '_'.join(sequence_info)+'.nii'
    return sequence_name

#TO-DO: T1 has 3D, 2D

In [502]:
# Naming to be established:
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
def filt_T2(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    
    ignore=['AVG', 'MIN', 'MIP']
    if any(word in sequence_name for word in ignore):
        return None
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        return None
#     print(sequence_info)
        
    return '_'.join(sequence_info)+'.nii'

#TO-DO: T2 has difference pulse types :0 and some other things

In [503]:
def filt_diffusion(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

#TO-DO: this one will be A LOT OF FUN!

In [504]:
# BOLD naming convention: <ID>_BOLD_<DOE>.nii
def filt_bold(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    
    problem_bold = None
    
    sequence_info = ignore_notes(sequence_info)
    sequence_name = '_'.join(sequence_info)+'.nii'    
    
    # The following ID's are the problem ones (from previous runs). Checking if these ID's are unique
#     probs = ['SH005', 'WH1296', 'MEN009', 'WH1612', 'WH1268', 'MEN040', 'SH019', 
#              'MEN023', 'MEN007', 'WHITTIER5002']
    
    # If it contains EQ_1 or a letter attached to the end of the DOE,
    # then there is a previous one that already exists
    if sequence_info[-2] == 'EQ' or not sequence_info[-1].isdigit():
        return None
        
#     elif ID in probs:
#         problem_bold = sequence_name
    
    elif len(sequence_info) != 3:
        problem_bold = sequence_name
    
#     return problem_bold
    return sequence_name

In [505]:
def filt_asl(sequence_name):
    sequence_info=get_sequence_info(sequence_name)            
    sequence_info=ignore_notes(sequence_info)
    if '3D' in sequence_info and not sequence_info[-1].isdigit():
        return None
    return '_'.join(sequence_info)+'.nii'

In [506]:
def filt_csf(sequence_name):
    if '1.5' in sequence_name:
        return None
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(sequence_info)
    if not sequence_info[-1].isdigit():
        return None
    return '_'.join(sequence_info)+'.nii'

#TO-DO: CSF of different location and CINE ON/OFF

In [507]:
def filt_carotid(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [508]:
def filt_aorta(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    if sequence_info[2] != 'AORTA':
        return None
    return '_'.join(sequence_info)+'.nii'

In [509]:
def filt_candy_cane(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    
    if not sequence_info[-1].isdigit():
        return None
    else:
        return '_'.join(sequence_info)+'.nii'

In [510]:
def filt_arch(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [511]:
def filt_mpr(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [512]:
def filt_cbf(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    if sequence_info[-2] == 'EQ' or not sequence_info[-1].isdigit():
        print(sequence_info)
#     print(sequence_info)

In [513]:
def filt_probe(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(sequence_info)
    return sequence_name[:-4]+'.nii'

In [514]:
def get_path(root):
    
    t1=[]
    bold = []
    flair = []
    flair_3d = []
    flair_2d = []
    t2 = []
    asl=[]
    candy_cane=[]
    aorta=[]
    
    csf = []
    probe = []
    change = 0
    count = 0
    
    for path, subdirs, files in os.walk(root):
        for file in files:
            file_upper=base_standard(file.upper())
            if fnmatch.fnmatch(file_upper, '*T1*NII') or fnmatch.fnmatch(file_upper, '*FSPGR*NII'):                 
                n_file=filt_T1(file_upper)
#                 print("T1:", n_file)
                if n_file is not None:
                    t1.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*FLAIR*NII'): 
#                 print("FLAIR: \t", file_upper)
                n_file = filt_FLAIR(file_upper, flair_3d, flair_2d)
#                 print("FLAIR:", n_file)
                if n_file is not None:
                    flair.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*T2*NII'):
#                 print('\t', file_upper)
                n_file = filt_T2(file_upper)
                if n_file is not None:
                    t2.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*BOLD*NII'):
                n_file = filt_bold(file_upper)
                if n_file is not None:
#                     print(file, '-->', n_file)
                    bold.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*DTI*NII') or fnmatch.fnmatch(file_upper, '*DKI*NII') or fnmatch.fnmatch(file_upper, '*DIFF*NII') or fnmatch.fnmatch(file_upper, '*DWI*NII') or fnmatch.fnmatch(file_upper, '*AVER*DC*NII') or fnmatch.fnmatch(file_upper, '*FRACT*ANISO*NII'):
#                 print('\t', file_upper)
#                 sort_diffusion(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*ASL*NII'):
                n_file=filt_asl(file_upper)
                if n_file is not None:
                    asl.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*SWAN*NII'):
                # Swan files has very consistant sequence naming
                # beaware of ID naming though
                n_file=file_upper.replace('NII', 'nii')
#                 #TODO: rename file
#                 print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*CSF*NII'):
#                 print('CSF:', file_upper)
                n_file = filt_csf(file_upper)
                if n_file is not None:
                    csf.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
                pass
            elif fnmatch.fnmatch(file_upper, '*CAROTID*NII'):
                #Need to ask what the carotid is doing: 
                #1. need to know the naming convention for difference carotids
#                 print('Carotid:', file_upper)
#                 filt_carotid(file_upper)
            elif fnmatch.fnmatch(file_upper, '*AORTA*NII'):
#                 count += 1
#                 print(file_upper)
#                 filt_aorta(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*AORTA*NII'):
                n_file=filt_aorta(file_upper)
                if (n_file is not None) and (file != n_file):
                    aorta.append((file, n_file))
                #TODO: rename file
            elif fnmatch.fnmatch(file_upper, '*CEREBR*BLOOD*FLOW*NII'):
#                 print(file_upper)
                n_file=filt_cbf(file_upper)
#                 filt_cbf(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*MPR*NII'):
#                 print(file_upper)
#                 filt_mpr(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*CANDY_CANE*NII'):
#                 print('Candy cane:', file_upper)
                n_file=filt_candy_cane(file_upper)
                if n_file is not None:
                    candy_cane.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*ARCH*NII'):
                #    
                # 1. the inconsidency happens at the missing venc number.
                #
#                 print(file_upper)
#                 filt_arch(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*PROBE*NII'):
#                 print(file_upper)
                n_file = filt_probe(file_upper)
                if n_file is not None:
                    asl.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
                pass
            elif fnmatch.fnmatch(file_upper, '*NII'):
#                 print('\t', file_upper)
                pass
            else:
#                 print('\t\t', file_upper)
                pass

#     print(len(T1))
#     print("count: ", count)
#     print(len(asl))
#     print(len(bolds))
#     for pair in bolds:
#         print(pair)
#     print(len(flair))
#     print(len(flair_3d))
#     print(len(flair_2d))
#     print(len(t2))
#     for pair in t2:
#         print(pair)
#     print(change)
#     print(len(csf))
#     for pair in csf:
#         print(pair)

In [515]:
def main():
#     source_dir=input("Enter source dir:")
#     source_dir="/Users/baymac/Desktop/Project_NaCl_Test_DataSet/3T_Last_updated_31OCT19KW"
    source_dir = '/media/ke/8tb_part2/FSL_work/all_info'
    get_path(source_dir)
        

In [516]:
if __name__ =="__main__":
    main()

['WH1615', 'CEREBRAL', 'BLOOD', 'FLOW', '20180105140227B']
['WH1615', 'CEREBRAL', 'BLOOD', 'FLOW', '20180105140227A']
['WH1615', 'CEREBRAL', 'BLOOD', 'FLOW', '20180105140227B', 'EQ', '1']
